In [1]:
import sys
sys.path.append("../") # go to parent dir



In [2]:
# imports
import simpy
import time
from cluster_simulator.cluster import Cluster, Tier, bandwidth_share_model, compute_share_model, get_tier, convert_size
from cluster_simulator.phase import DelayPhase, ComputePhase, IOPhase
from cluster_simulator.application import Application
from cluster_simulator.analytics import display_run

In [3]:
# preparing execution environment variables
env = simpy.Environment()
data = simpy.Store(env)
app1 = Application(env, name="app1", # name of the app in the display
                   compute=[0, 15],  # two events, first at 0 and second at 15, and compute between them
                   read=[3e9, 0],    # read 3GB at 0, before compute phase, at the end do nothing (0)
                   write=[0, 5e9],  # write 0GB at first event, and 10GB at the second, after compute phase
                   data=data)        # collected data for monitoring

# app1 : read 1GB -> compute 15s if 1 core is used -> write 10GB

ssd_bandwidth =   {'read':  {'seq': 210, 'rand': 190}, # throughput for read ops in MB/s
                       'write': {'seq': 100, 'rand': 100}} # for read/write random/sequential I/O

# register the tier with a name and a capacity
ssd_tier = Tier(env, 'SSD', bandwidth=ssd_bandwidth, capacity=200e9)
hdd_bandwidth = {'read':  {'seq': 80, 'rand': 80},
                    'write': {'seq': 40, 'rand': 40}}

# register the tier with a name and a capacity
hdd_tier = Tier(env, 'HDD', bandwidth=hdd_bandwidth, capacity=1e12)
# we attach the tiers to a cluster:
cluster = Cluster(env, compute_nodes=3,   # number of physical nodes
                       cores_per_node=2,  # available cores per node
                       tiers=[hdd_tier, ssd_tier]) # associate storage tiers to the cluster
                    #          ^tier 0,    ^tier 1, tier...
                    
start_time = time.time()
env.process(app1.run(cluster, placement=[0, 1]))
env.run()
print(f"Execution time = {time.time()-start_time} seconds")

fig = display_run(data, cluster, width=800, height=800)
fig.show()
fig.write_html("../docs/figure.html")

2022-06-13 16:55:57.758 | DEBUG    | cluster_simulator.phase:monitor_step:43 - 
 | Monitoring| app: app1 | type: read | cpu_usage: 1 | t_start: 0 | t_end: 37.5 | bandwidth_concurrency: 1 | bandwidth: 80.0 MB/s | phase_duration: 37.5 | volume: 3.0 GB | tiers: ['HDD', 'SSD'] | data_placement: {'placement': 'HDD'} | init_level: {'HDD': 0, 'SSD': 0} | tier_level: {'HDD': 3000000000.0, 'SSD': 0} 
2022-06-13 16:55:57.763 | DEBUG    | cluster_simulator.application:run:197 - the issued status of the IO phase : True
2022-06-13 16:55:57.764 | INFO     | cluster_simulator.phase:run:124 - (App app1) - Start computing phase at 37.5 with 1 requested cores
2022-06-13 16:55:57.765 | DEBUG    | cluster_simulator.phase:monitor_step:43 - 
 | Monitoring| app: app1 | type: compute | cpu_usage: 1 | t_start: 37.5 | t_end: 52.5 | bandwidth: 0 MB/s | phase_duration: 15.0 | volume: 0B | tiers: ['HDD', 'SSD'] | data_placement: None | init_level: {'HDD': 3000000000.0, 'SSD': 0} | tier_level: {'HDD': 3000000000.0,

Execution time = 0.012043952941894531 seconds
